### Update Q value by 3 samples, using ϵ-greedy Q with ϵ=0.25, α=0.8.

In [28]:
import random
import numpy as np

# Parameters
epsilon = 0.25
alpha = 0.8
gamma = 1.0
num_steps = 3
states = ['A', 'B', 'C', 'D', 'E']
terminal_states = ['A', 'E']
actions = [0, 1]  # 0: Left, 1: Right

# Rewards
rewards = {
    'A': 1000,
    'E': 10,
    'B': -1,
    'C': -1,
    'D': -1
}

# Initialize Q(S, A) using weights w = [1, 1, -1] for linear approximation
weights = np.array([1.0, 1.0, -1.0])  # Initial weights for [distance to left wall, turn direction, bias]

# Feature extraction function
def feature_vector(state, action):
    # Distance to the left wall
    left_wall_dist = states.index(state)
    
    # Turn direction: +1 for left (action=0), -1 for right (action=1)
    turn_dir = 1 if action == 0 else -1
    
    # Bias term
    bias = 1
    
    # Feature vector: [distance, turn direction, bias]
    return np.array([left_wall_dist, turn_dir, bias])

# Calculate Q-value using linear function approximation
def q_value(state, action):
    x_sa = feature_vector(state, action)
    return np.dot(weights, x_sa)

# epsilon-greedy policy
def epsilon_greedy(state):
    if random.random() < epsilon:
        return random.choice(actions)  # Explore (random action)
    else:
        # Exploit: select action with the highest Q-value
        q_vals = [q_value(state, a) for a in actions]
        return np.argmax(q_vals)

# Transition function (deterministic)
def transition(state, action):
    if state == 'D':
        return 'C' if action == 0 else 'E'
    elif state == 'C':
        return 'B' if action == 0 else 'D'
    elif state == 'B':
        return 'A' if action == 0 else 'C'
    return state  # A and E are terminal, no transition

# Main Q-learning with 3 samples
state = 'D'  # Start at initial state D

for _ in range(num_steps):
    # Select action using epsilon-greedy policy
    action = epsilon_greedy(state)
    
    # Get the next state and reward
    next_state = transition(state, action)
    reward = rewards[next_state]
    
    # Calculate TD target
    if next_state in terminal_states:
        td_target = reward
    else:
        max_next_q = max([q_value(next_state, a) for a in actions])  # Best Q-value for next state
        td_target = reward + gamma * max_next_q
    
    # Update Q-value using linear approximation (update weights)
    q_current = q_value(state, action)
    td_error = td_target - q_current
    weights += alpha * td_error * feature_vector(state, action)
    
    # Move to the next state
    state = next_state
    
    # Output current Q-values
    print(f"Step {_ + 1}:")
    print(f"State: {state}, Action: {action}, Reward: {reward}, Weights: {weights}\n")

# Final Q-values
print("Final weights after 3 samples:", weights)


Step 1:
State: C, Action: 0, Reward: -1, Weights: [-3.8 -0.6 -2.6]

Step 2:
State: D, Action: 1, Reward: -1, Weights: [-11.48   3.24  -6.44]

Step 3:
State: C, Action: 0, Reward: -1, Weights: [13.672 11.624  1.944]

Final weights after 3 samples: [13.672 11.624  1.944]


### What is the optimal weight value w look like? (Hint: you can train until convergence, or give some explanation of the optimal weight value w)

In [35]:
import random
import numpy as np

# Parameters
epsilon = 0.25
alpha = 0.8
gamma = 1.0
convergence_threshold = 1e-6  # Stop if weight changes are smaller than this
max_iterations = 1000  # Maximum iterations for training
states = ['A', 'B', 'C', 'D', 'E']
terminal_states = ['A', 'E']
actions = [0, 1]  # 0: Left, 1: Right

# Rewards
rewards = {
    'A': 1000,
    'E': 10,
    'B': -1,
    'C': -1,
    'D': -1
}

# Initialize Q(S, A) using weights w = [1.0, 1.0, -1.0] for linear approximation
weights = np.array([1.0, 1.0, -1.0])

# Feature extraction function
def feature_vector(state, action):
    # Distance to the left wall
    left_wall_dist = states.index(state)
    
    # Turn direction: +1 for left (action=0), -1 for right (action=1)
    turn_dir = 1 if action == 0 else -1
    
    # Bias term
    bias = 1
    
    # Feature vector: [distance, turn direction, bias]
    return np.array([left_wall_dist, turn_dir, bias])

# Calculate Q-value using linear function approximation
def q_value(state, action):
    x_sa = feature_vector(state, action)
    return np.dot(weights, x_sa)

# epsilon-greedy policy
def epsilon_greedy(state):
    if random.random() < epsilon:
        return random.choice(actions)  # Explore (random action)
    else:
        # Exploit: select action with the highest Q-value
        q_vals = [q_value(state, a) for a in actions]
        return np.argmax(q_vals)

# Transition function (deterministic)
def transition(state, action):
    if state == 'D':
        return 'C' if action == 0 else 'E'
    elif state == 'C':
        return 'B' if action == 0 else 'D'
    elif state == 'B':
        return 'A' if action == 0 else 'C'
    return state  # A and E are terminal, no transition

# Train Q-learning until convergence
def train_until_convergence():
    global weights
    state = 'D'  # Start at initial state D
    for iteration in range(max_iterations):
        # Store old weights for comparison
        old_weights = weights.copy()
        
        # Select action using epsilon-greedy policy
        action = epsilon_greedy(state)
        
        # Get the next state and reward
        next_state = transition(state, action)
        reward = rewards[next_state]
        
        # Calculate TD target
        if next_state in terminal_states:
            td_target = reward
        else:
            max_next_q = max([q_value(next_state, a) for a in actions])  # Best Q-value for next state
            td_target = reward + gamma * max_next_q
        
        # Update Q-value using linear approximation (update weights)
        q_current = q_value(state, action)
        td_error = td_target - q_current
        weights += alpha * td_error * feature_vector(state, action)  # Update weights
        
        # Move to the next state
        state = next_state
        
        # Check for convergence (if weight changes are smaller than threshold)
        if np.linalg.norm(weights - old_weights) < convergence_threshold:
            print(f"Converged after {iteration + 1} iterations.")
            break

train_until_convergence()

# Output final weights
print("Optimal weight vector:", weights)

Converged after 56 iterations.
Optimal weight vector: [815.04136858  24.04616771 975.95383177]


### Represent the value function by two vectors, w1 for turn left, w2 for turn right, what is the optimal weight value in this case.

In [31]:
import random
import numpy as np

# Parameters
epsilon = 0.25
alpha = 0.01
gamma = 1.0
convergence_threshold = 1e-6  # Stop if weight changes are smaller than this
max_iterations = 1000  # Maximum iterations for training
states = ['A', 'B', 'C', 'D', 'E']
terminal_states = ['A', 'E']
actions = [0, 1]  # 0: Left, 1: Right

# Rewards
rewards = {
    'A': 1000,
    'E': 10,
    'B': -1,
    'C': -1,
    'D': -1
}

# Initialize Q(S, A) with separate weight vectors w1 for Left and w2 for Right
w1 = np.array([1.0, 1.0, -1.0])  # Weights for left action (0)
w2 = np.array([1.0, 1.0, -1.0])  # Weights for right action (1)

# Feature extraction function
def feature_vector(state, action):
    # Distance to the left wall
    left_wall_dist = states.index(state)
    
    # Turn direction: +1 for left (action=0), -1 for right (action=1)
    turn_dir = 1 if action == 0 else -1
    
    # Bias term
    bias = 1
    
    # Feature vector: [distance, turn direction, bias]
    return np.array([left_wall_dist, turn_dir, bias])

# Calculate Q-value using linear function approximation for both actions
def q_value(state, action):
    x_sa = feature_vector(state, action)
    if action == 0:
        return np.dot(w1, x_sa)
    else:
        return np.dot(w2, x_sa)

# epsilon-greedy policy
def epsilon_greedy(state):
    if random.random() < epsilon:
        return random.choice(actions)  # Explore (random action)
    else:
        # Exploit: select action with the highest Q-value
        q_vals = [q_value(state, a) for a in actions]
        return np.argmax(q_vals)

# Transition function (deterministic)
def transition(state, action):
    if state == 'D':
        return 'C' if action == 0 else 'E'
    elif state == 'C':
        return 'B' if action == 0 else 'D'
    elif state == 'B':
        return 'A' if action == 0 else 'C'
    return state  # A and E are terminal, no transition

# Train Q-learning until convergence
def train_until_convergence():
    global w1, w2
    state = 'D'  # Start at initial state D
    for iteration in range(max_iterations):
        # Store old weights for comparison
        w1_old = w1.copy()
        w2_old = w2.copy()
        
        # Select action using epsilon-greedy policy
        action = epsilon_greedy(state)
        
        # Get the next state and reward
        next_state = transition(state, action)
        reward = rewards[next_state]
        
        # Calculate TD target
        if next_state in terminal_states:
            td_target = reward
        else:
            max_next_q = max([q_value(next_state, a) for a in actions])  # Best Q-value for next state
            td_target = reward + gamma * max_next_q
        
        # Update Q-value using linear approximation (update weights)
        q_current = q_value(state, action)
        td_error = td_target - q_current
        
        if action == 0:
            w1 += alpha * td_error * feature_vector(state, action)  # Update w1 for left action
        else:
            w2 += alpha * td_error * feature_vector(state, action)  # Update w2 for right action
        
        # Move to the next state
        state = next_state
        
        # Check for convergence (if weights change is smaller than threshold)
        if np.linalg.norm(w1 - w1_old) < convergence_threshold and np.linalg.norm(w2 - w2_old) < convergence_threshold:
            print(f"Converged after {iteration + 1} iterations.")
            break


train_until_convergence()

# Output final weights for w1 and w2
print("Optimal weight vector for Left (w1):", w1)
print("Optimal weight vector for Right (w2):", w2)

Converged after 943 iterations.
Optimal weight vector for Left (w1): [ 10.892976   500.99996537 498.99996537]
Optimal weight vector for Right (w2): [   1.         -459.90435654  459.90435654]
